In [1]:
from enron import create_dataset_from_datalist

ds, size_ds = create_dataset_from_datalist()


Working with enron0.oaiur.mongodb.net/EnronDB
----------------------------------------------
Start copying cursor
Progress: |██████████████████████████████████████████████████| 100.00% Complete (130597/130597)
Cursor copied
Starting process...
Progress: |██████████████████████████████████████████████████| 100.00% Complete (130597/130597)

Working with enron1.mymux.mongodb.net/EnronDB
----------------------------------------------
Start copying cursor
Progress: |██████████████████████████████████████████████████| 100.00% Complete (78039/78039)
Cursor copied
Starting process...
Progress: |██████████████████████████████████████████████████| 100.00% Complete (78039/78039)

Working with enron2.2gdr4.mongodb.net/EnronDB
----------------------------------------------
Start copying cursor
Progress: |██████████████████████████████████████████████████| 100.00% Complete (33800/33800)
Cursor copied
Starting process...
Progress: |██████████████████████████████████████████████████| 100.00% Complete

In [2]:
print(f'Se han procesado {size_ds} mensajes')

Se han procesado 231222 mensajes


In [6]:
for e in ds.take(3):
    print(e)

tf.Tensor(
[b'they' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b''
 b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b''
 b'' b'' b'' b'' b'can' b'be' b'found' b'' b'' b'sheet' b'deals' b'' b''
 b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b''
 b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b''
 b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b''
 b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b''
 b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'I' b'' b''
 b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b''
 b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b''
 b'did' b'' b'' b'' b'' b'sheet' b'deals' b'' b'' b'' b'' b'' b'' b'' b''
 b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b''
 b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b''
 b'' b'' b'' b'' b'' b'' b'' b'' b'' b

In [8]:
import tensorflow as tf
from enron import tokenize_svo_and_body

model_name = "ted_hrlr_translate_pt_en_converter"
tf.keras.utils.get_file(
    f"{model_name}.zip",
    f"http://storage.googleapis.com/download.tensorflow.org/models/{model_name}.zip",
    cache_dir='.', cache_subdir='', extract=True
)
tokenizers = tf.saved_model.load(model_name)
ds1 = ds.cache().map(lambda x: tokenize_svo_and_body(tokenizers, x), num_parallel_calls=tf.data.AUTOTUNE)

In [19]:
def remove_last_zeros(t):
    while tf.size(t) > 0 and t[-1] == 0:
        t = t[:-1]
    return t

for e in ds1.take(3):
    print(tf.shape(remove_last_zeros(e[0]))[0])
    print(tf.shape(remove_last_zeros(e[1]))[0])
    print(remove_last_zeros(e[0]))
    print(remove_last_zeros(e[1]))
    
len(ds1)

tf.Tensor(45306, shape=(), dtype=int32)
tf.Tensor(56, shape=(), dtype=int32)
tf.Tensor([ 2 83  3 ...  0  2  3], shape=(45306,), dtype=int64)
tf.Tensor(
[   2   45  149  147    1  266   14   77  787  175  743  725   88    1
  208   72   83   94   97  280   77  154 2204   55  826  227  217 5519
   56 6517   77   71   77  787  175  743 3116  227   15 1102 2823  116
   77   71   55  899  117   76   80  520   15 1120   13   54 6021    3], shape=(56,), dtype=int64)
tf.Tensor(99509, shape=(), dtype=int32)
tf.Tensor(123, shape=(), dtype=int32)
tf.Tensor([ 2 79  3 ...  0  2  3], shape=(99509,), dtype=int64)
tf.Tensor(
[   2   45 1081   37 1597  200 6138   52 1107  376  566   72   76   80
   77   71   46  608   17 1222 6138   52 1107  376 3116  227  466  664
 6138   52 1107  376  566   15   71 1775   86  112  361 2855   74   71
   52 1394   46 2244   72 2498   77  531   73 2577   77   71   40 1058
 4312 6542   77 1192  376   10 3251   72 4237  872   16 4773 1674   11
   15   76  234  161   76 14

231222

In [22]:
max_svo = tf.constant(0, dtype = tf.int32)
max_body = tf.constant(0, dtype = tf.int32)

def get_max_length(old, new_s):
    e0 = remove_last_zeros(new_s[0])
    e1 = remove_last_zeros(new_s[1])
    return tf.math.maximum(tf.shape(e0)[0], old[0]), tf.math.maximum(tf.shape(e1)[0], old[1])

print(ds1.cache().reduce((max_svo, max_body), get_max_length))

ResourceExhaustedError: OOM when allocating tensor with shape[171718] and type int64 on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node while/body/_1/while/strided_slice}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:ReduceDataset]

In [ ]:
def remove_all_zeros(t):
    t = remove_last_zeros(t)
    p = tf.argmin(t)
    m = tf.min(t)
    while m == 0:
        t = tf.reshape([t[:p], t[p+1:]], [-1])
        p = tf.argmin(t)
        m = tf.min(t)
    return t

for e in ds1.take(3):
    print(tf.shape(remove_all_zeros(e[0]))[0])
    print(tf.shape(remove_all_zeros(e[1]))[0])
    print(remove_all_zeros(e[0]))
    print(remove_all_zeros(e[1]))
    
len(ds1)

In [ ]:
max_svo = tf.constant(0, dtype = tf.int32)
max_body = tf.constant(0, dtype = tf.int32)

for e in ds1:
    e0 = remove_all_zeros(e[0])
    e1 = remove_all_zeros(e[1])
    if tf.shape(e0)[0] > max_svo:
        max_svo = tf.shape(e0)[0]
    if tf.shape(e1)[0] > max_body:
        max_body = tf.shape(e1)[0]
print(max_svo)
print(max_body)